In [5]:
!pip install pandas


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import pandas as pd

In [7]:
questions = pd.read_csv("questions_fiches_fip.csv")
questions.head(5)

,Unnamed: 0,question,num_texte
0,792,Quelle obligation subsiste pour un contrat de ...,72
1,328,Que se passe-t-il si l’avis d’impôt est reçu e...,32
2,614,Dans quel délai doit-on déposer une déclaratio...,58
3,159,Comment un entrepreneur relevant du régime rée...,15
4,967,Quelle condition relative à l'occupation doit ...,83


In [8]:
print(questions.shape)

(1427, 3)


In [9]:
info = pd.read_csv("info_particulier_impot.csv")
info.head()

,Unnamed: 0,niveau_0,niveau_1,URL,Titre,Texte
0,0,https://www.impots.gouv.fr/particulier/declare...,https://www.impots.gouv.fr/particulier/je-decl...,https://www.impots.gouv.fr/particulier/lobliga...,OBLIGATIONS DÉCLARATIVES,Obligations déclaratives\nL'obligation de décl...
1,1,https://www.impots.gouv.fr/particulier/declare...,https://www.impots.gouv.fr/particulier/je-decl...,https://www.impots.gouv.fr/particulier/modalit...,MODALITÉS DE DÉCLARATION,Modalités de déclaration\nLe principe : une se...
2,2,https://www.impots.gouv.fr/particulier/declare...,https://www.impots.gouv.fr/particulier/je-decl...,https://www.impots.gouv.fr/particulier/declare...,DÉCLAREZ EN LIGNE,Déclarez en ligne\nLes avantages de la déclara...
3,3,https://www.impots.gouv.fr/particulier/declare...,https://www.impots.gouv.fr/particulier/je-decl...,https://www.impots.gouv.fr/particulier/votre-s...,VOTRE SITUATION DE FAMILLE,Votre situation de famille\nPrélèvement à la s...
4,4,https://www.impots.gouv.fr/particulier/declare...,https://www.impots.gouv.fr/particulier/je-decl...,https://www.impots.gouv.fr/particulier/enfants...,ENFANTS MINEURS,Enfants mineurs\nPrélèvement à la source : afi...


In [10]:
info.shape

(113, 6)

In [11]:
info["Texte"]

0      Obligations déclaratives\nL'obligation de décl...
1      Modalités de déclaration\nLe principe : une se...
2      Déclarez en ligne\nLes avantages de la déclara...
3      Votre situation de famille\nPrélèvement à la s...
4      Enfants mineurs\nPrélèvement à la source : afi...
                             ...                        
108    Délais de réclamation\nPour l'impôt sur le rev...
109    Suivre ma réclamation\nComment suivre ma récla...
110    Le conciliateur départemental\nUn différend vo...
111    Le médiateur du Ministère\nL’institution du mé...
112    Délai de paiement\nSi vous avez des difficulté...
Name: Texte, Length: 113, dtype: object

In [12]:
from dotenv import load_dotenv
load_dotenv()

True

In [13]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

hf_token = os.environ.get("HUGGINGFACE_TOKEN", True)
model_id = "mistralai/Mistral-7B-Instruct-v0.3"

# Charge tokenizer (en mode slow si tu préfères)
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    trust_remote_code=True,
    token=hf_token,
    use_fast=False
)

# Charge le modèle SANS device_map
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,   # ou torch.float32 si pas de GPU compatible
    trust_remote_code=True,
    token=hf_token
)

# Déplace le modèle sur CUDA (ou CPU si pas de GPU)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Prépare le pipeline
chat = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0 if device=="cuda" else -1,  # 0 pour le GPU, -1 pour CPU
    max_new_tokens=200,
    temperature=0.7,
    do_sample=True,
    token=hf_token
)

# Génère une réponse
prompt = "[INST] Explique-moi le machine learning en deux phrases. [/INST]"
print(chat(prompt)[0]["generated_text"])


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.

KeyboardInterrupt



In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

try:
    tokenizer = AutoTokenizer.from_pretrained(
        model_id,
        trust_remote_code=True,
        token=hf_token,
        use_fast=False,
        local_files_only=True         # <— force le mode offline
    )
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        trust_remote_code=True,
        token=hf_token,
        local_files_only=True         # <— idem
    )
    print("Chargé depuis le cache.")
except Exception as e:
    print("Modèle/tokenizer non trouvés en cache :", e)
    # Ici tu peux soit :
    # 1. Retenter sans local_files_only pour déclencher le téléchargement,
    # 2. Inviter l’utilisateur à lancer `huggingface-cli login` et réessayer.


In [ ]:

# Déplace le modèle sur CUDA (ou CPU si pas de GPU)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Prépare le pipeline
chat = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0 if device=="cuda" else -1,  # 0 pour le GPU, -1 pour CPU
    max_new_tokens=200,
    temperature=0.7,
    do_sample=True,
    token=hf_token
)

# Génère une réponse
prompt = "[INST] Explique-moi le machine learning en deux phrases. [/INST]"
print(chat(prompt)[0]["generated_text"])
